Ensure that our database is ready

In [ ]:
%%bash
if [[ -d project-tycho-utilities ]];
then
  cd project-tycho-utilities/
  git pull
else
  git clone https://github.com/lgautier/project-tycho-utilities.git
  cd project-tycho-utilities/
fi
DBNAME=../tycho.db make all

---


<!-- label:sqlite -->

Opening a connection to a database (here an SQLite database)
and getting a cursor is straightforward.

In [ ]:
import sqlite3
dbfilename = "tycho.db"
dbcon = sqlite3.connect(dbfilename)
cursor = dbcon.cursor()

<!-- label:sqlite_firstquery -->

Our first query is simple: we want to fetch the cities
in states with a name starting with "M"

In [ ]:
sql = """
SELECT state, city
FROM location
WHERE state LIKE 'M%'
"""
cursor.execute(sql)

<!-- label:sqlite_firstresults -->
Results can then be pulled from the database, and further
computation done with Python.

---

Results can subsequently be fetched with:

In [ ]:
cursor.fetchmany(10)

Notes:
- Databases can hold a LARGE amount of data. The following statement is potentially retrieving a LARGE
amount of data. Use with care.
- The following command will retrieve remaining results after the call to `fetchmany()` above.

In [ ]:
all_the_rest = cursor.fetchall()

In [ ]:
# clean up
del(all_the_rest)

---

If what we want is to count the number of cities in each state
matching our predicate, this can be achieved with:

In [ ]:
cursor.execute(sql)

from collections import Counter
ct = Counter(row[0] for row in cursor)

ct

<!-- label:sqlite_secondquery -->
Some of the post-processing done in Python can be pushed
back to the database

In [ ]:
sql = """
SELECT state, count(city) AS ct
FROM location
WHERE state LIKE 'M%'
GROUP BY state
ORDER BY ct DESC
"""

cursor.execute(sql)

cursor.fetchmany(10)

There is usually documentation about the database schema, but querying the database
to ask information about a given table can be convenient. This is less standardized
SQL, here is how to achieve this with sqlite:

In [ ]:
def table_info(tablename, cursor):
    sql = 'PRAGMA table_info(%s)' % tablename
    cursor.execute(sql)
    return cursor.fetchall()

In [ ]:
for tablename in ("location", "casecount", "disase"):
  print(tablename)
  print(table_info(tablename, cursor))
  print()


Results can be further exported to different formats.

In [ ]:
sql = """
SELECT
  state, city
FROM
  location
"""

cursor.execute(sql)

import csv
with open('location.csv', 'w') as fh:
  csv_w = csv.writer(fh)
  csv_w.writerow(('state', 'city'))
  csv_w.writerows(cursor)

---

Our database schema has an other table `casecount` that contains
the count of cases for several diseases broken down by city and date.

Now we want the answer to a slightly more complex question: for each state,
count the number of cities for which we have cases
for more than 5 distinct diseases. Oh, and sort the list of states in decreasing
number of such diseases. In fact, only report the first 5 states.

<!-- label:sqlite_complexquery -->

In [ ]:
sql = """
SELECT state, count(city) city_count
FROM (SELECT deadly_cases.location_id
      FROM (SELECT location_id, COUNT(DISTINCT(disease_id)) AS d_count
            FROM casecount
            WHERE event="CASES"
            GROUP BY location_id) AS deadly_cases
      WHERE deadly_cases.d_count > 5) AS selected_locations
INNER JOIN location
ON selected_locations.location_id = location.id
GROUP BY state
ORDER BY city_count DESC
LIMIT 5
"""
cursor.execute(sql)

---

<!-- label:sqlalchemy_open -->
Opening the same database using an ORM (SQLalchemy).

In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

Base = automap_base()

# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine("sqlite:///tycho.db")

<!-- label:sqlalchemy_reflect -->
Use reflection on the SQL side to create the objects from the database.

In [ ]:
Base.prepare(engine, reflect=True)
location = Base.classes.location

---

<!-- label:sqlalchemy_query -->
Make a query using SQLalchemy's methods.

In [ ]:
session = Session(engine)
from sqlalchemy import func # SQL functions

query = (session
         .query(location.state,
                func.count(location.city))
         .filter(location.state.like('M%'))
         .group_by(location.state)
         .order_by(func.count(location.city).desc())
         .limit(5))
res = query.all()


Note that SQL and ORM mapping are technology predating StackOverflow.

---

<!-- label:sqlalchemy_sql -->
Function composition is generating SQL code.

In [ ]:
from sqlalchemy.dialects import sqlite
sql = query.statement.compile(dialect=session.bind.dialect)
print(str(sql).replace('GROUP BY', '\nGROUP BY'))

---

## Exercises:

Can you answer the following in SQL or with SQLAlchemy ?

- Count the number of cities in states with a name starting with 'N'

- Count the number of cities in each state.

- Count the number of cities with a name starting with 'N', stratified by state.

- for each state,
  count the number of cities for which we have deadly cases
  for more than 5 distinct diseases. Oh, and sort the list of states in decreasing
  number of such diseases. In fact, only report the first 10 states. (hint: this is
  pretty much the last example query about).

- Count the total number of cases of flu in NYC
  (hint: flu is a short name, you may want the long name)

- Count the number of cases of flu in NYC each year

---